## metrics

In [1]:
import json
import collections
import pandas as pd
import numpy as np
english_exp_names_1 = []
# english_exp_names_3 = ['english', 'english_stopword', 'english_delete_5', 'english_delete_10', \
#         'english_insert_5', 'english_insert_10', \
#         'xlingualreplace_5', 'english_replace_10', 'xlingualshuffle_words', \
#         'english_repeat_sentences', 'english_shuffle_sentences', 'english_shuffle_instruction']
english_exp_names_3 = ['english_original', 'english_delete_stopwords', 'english_delete_10_words', \
        'english_insert_10_words', 'english_replace_10_words', 'english_shuffle_words', \
        'english_repeat_sentences', 'english_shuffle_sentences', 'english_shuffle_instructions']
exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'coreference_resolution', \
    'dialogue_act_recognition', 'word_analogy', 'textual_entailment']
non_classifications = ['data_to_text', 'grammar_error_correction', 'title_generation',\
    'keyword_tagging', 'overlap_extraction', 'question_rewriting']
for english_exp_name in english_exp_names_1:
    metric_file = open('./tk_outputs/output_{}/all_results.json'.format(english_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('./tk_outputs/output_{}/all_results.json'.format("english"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for english_exp_name in english_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in range(4, 7):
        _english_exp_name = english_exp_name + "_{}".format(index)

        metric_file = open('./tk_outputs/output_{}/all_results.json'.format(_english_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric_names:
            metric_samples[metric_name].append(metric[metric_name])
    
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
english_rougeL = pd.DataFrame(rougeL)
english_rougeL.index = english_exp_names_1 + english_exp_names_3
english_exact_match = pd.DataFrame(exact_match)
english_exact_match.index = english_exp_names_1 + english_exp_names_3

def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
print(english_rougeL.columns)
english_exact_match.columns = generate_new_columns(english_exact_match.columns)
english_rougeL.columns = generate_new_columns(english_rougeL.columns)

Index(['data_to_text', 'grammar_error_correction', 'title_generation',
       'keyword_tagging', 'overlap_extraction', 'question_rewriting',
       'eval_rougeL'],
      dtype='object')


In [5]:
print(english_exact_match.iloc[:, :-1].to_latex())
print(english_rougeL.iloc[:, :-1].to_latex())

\begin{tabular}{lllllll}
\toprule
{} &        AC &       CEC &        CR &       DAR &        WA &        TE \\
\midrule
english\_original             &  49.8±0.6 &  50.0±0.8 &  36.1±0.5 &  43.2±0.8 &  19.7±0.7 &  40.1±0.8 \\
english\_delete\_stopwords     &  44.2±0.4 &  45.0±1.2 &  32.3±0.7 &  28.7±0.9 &   7.6±0.1 &  31.6±0.5 \\
english\_delete\_10\_words      &  45.3±3.3 &  40.8±4.3 &  32.5±0.4 &  34.0±2.6 &  18.6±2.6 &  34.9±3.1 \\
english\_insert\_10\_words      &  44.5±1.9 &  40.3±4.2 &  34.0±2.0 &  38.9±1.2 &  18.5±1.2 &  35.1±0.4 \\
english\_replace\_10\_words     &  48.7±1.0 &  45.9±1.6 &  31.3±1.9 &  35.9±1.8 &  18.2±0.6 &  33.8±1.7 \\
english\_shuffle\_words        &  20.1±2.7 &   4.2±2.5 &  18.1±2.0 &   7.8±1.6 &  14.2±2.0 &   8.6±0.5 \\
english\_repeat\_sentences     &  51.1±0.8 &  46.4±1.0 &  34.8±1.0 &  37.6±4.0 &  16.4±2.3 &  41.3±1.4 \\
english\_shuffle\_sentences    &  49.9±1.2 &  46.6±1.3 &  34.0±1.0 &  42.4±0.1 &  16.4±1.5 &  40.0±0.9 \\
english\_shuffle\_instruction

/tmp/ipykernel_54076/2916011428.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_exact_match.iloc[:, :-1].to_latex())
/tmp/ipykernel_54076/2916011428.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_rougeL.iloc[:, :-1].to_latex())


In [1]:
import json
import collections
import pandas as pd
import numpy as np
xlingual_exp_names_1 = []
# xlingual_exp_names_3 = ['xlingual', 'xlingual_stopword', 'xlingual_delete_5', 'xlingual_delete_10', \
#         'xlingual_insert_5', 'xlingual_insert_10', \
#         'xlingual_replace_5', 'xlingual_replace_10', \
#         'xlingual_repeat_sentences', 'xlingual_shuffle_sentences', 'xlingual_shuffle_words', 'xlingual_shuffle_instruction']
xlingual_exp_names_3 = ['xlingual_original', 'xlingual_delete_stopwords', 'xlingual_delete_10_words', \
        'xlingual_insert_10_words', 'xlingual_replace_10_words', 'xlingual_shuffle_words', \
        'xlingual_repeat_sentences', 'xlingual_shuffle_sentences', 'xlingual_shuffle_instructions']
exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'textual_entailment']
non_classifications = ['title_generation']
for xlingual_exp_name in xlingual_exp_names_1:
    metric_file = open('./tk_outputs/output_{}/all_results.json'.format(xlingual_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('./tk_outputs/output_{}/all_results.json'.format("xlingual"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for xlingual_exp_name in xlingual_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in range(4, 7):
        _xlingual_exp_name = xlingual_exp_name + "_{}".format(index)

        metric_file = open('./tk_outputs/output_{}/all_results.json'.format(_xlingual_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric_names:
            metric_samples[metric_name].append(metric[metric_name])
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
xlingual_rougeL = pd.DataFrame(rougeL)
xlingual_rougeL.index = xlingual_exp_names_1 + xlingual_exp_names_3
xlingual_exact_match = pd.DataFrame(exact_match)
xlingual_exact_match.index = xlingual_exp_names_1 + xlingual_exp_names_3


def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
xlingual_exact_match.columns = generate_new_columns(xlingual_exact_match.columns)
xlingual_rougeL.columns = generate_new_columns(xlingual_rougeL.columns)

In [2]:
print(pd.concat([xlingual_exact_match.iloc[:, :-1], xlingual_rougeL.iloc[:, :-1]], axis=1).to_latex())

\begin{tabular}{lllll}
\toprule
{} &         AC &       CEC &        TE &        TG \\
\midrule
xlingual\_original             &   76.3±3.4 &  43.1±0.7 &  19.7±1.4 &  10.3±2.0 \\
xlingual\_delete\_stopwords     &   69.0±4.3 &  20.8±0.6 &  10.1±0.2 &   5.7±2.2 \\
xlingual\_delete\_10\_words      &  56.3±20.8 &  28.9±2.2 &  11.2±4.9 &   9.7±1.6 \\
xlingual\_insert\_10\_words      &   72.0±2.2 &  35.5±1.4 &  17.6±3.4 &   9.5±0.6 \\
xlingual\_replace\_10\_words     &   68.3±0.5 &  39.7±1.5 &  17.8±3.1 &  10.5±1.6 \\
xlingual\_shuffle\_words        &    3.0±1.6 &   0.2±0.1 &   1.4±2.0 &   8.6±2.3 \\
xlingual\_repeat\_sentences     &   73.3±2.6 &  32.6±2.7 &  14.6±3.5 &   9.2±1.7 \\
xlingual\_shuffle\_sentences    &   69.7±3.1 &  34.8±1.7 &  17.3±0.9 &   5.8±1.7 \\
xlingual\_shuffle\_instructions &    0.0±0.0 &  14.9±1.7 &   0.0±0.0 &   0.6±0.9 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_61464/3841236022.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.concat([xlingual_exact_match.iloc[:, :-1], xlingual_rougeL.iloc[:, :-1]], axis=1).to_latex())


In [2]:
print(xlingual_exact_match.iloc[:, :-1].to_latex())
# print(xlingual_rougeL.to_latex())

\begin{tabular}{llll}
\toprule
{} &         AC &       CEC &        TE \\
\midrule
xlingual\_original             &   76.3±3.4 &  43.1±0.7 &  19.7±1.4 \\
xlingual\_delete\_stopwords     &   69.0±4.3 &  20.8±0.6 &  10.1±0.2 \\
xlingual\_delete\_10\_words      &  56.3±20.8 &  28.9±2.2 &  11.2±4.9 \\
xlingual\_insert\_10\_words      &   72.0±2.2 &  35.5±1.4 &  17.6±3.4 \\
xlingual\_replace\_10\_words     &   68.3±0.5 &  39.7±1.5 &  17.8±3.1 \\
xlingual\_shuffle\_words        &    3.0±1.6 &   0.2±0.1 &   1.4±2.0 \\
xlingual\_repeat\_sentences     &   73.3±2.6 &  32.6±2.7 &  14.6±3.5 \\
xlingual\_shuffle\_sentences    &   69.7±3.1 &  34.8±1.7 &  17.3±0.9 \\
xlingual\_shuffle\_instructions &    0.0±0.0 &  14.9±1.7 &   0.0±0.0 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_46235/2672335181.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(xlingual_exact_match.iloc[:, :-1].to_latex())


In [56]:
metrics = {'english_exact_match': english_exact_match, 'english_rougeL': english_rougeL, \
    'xlingual_exact_match': xlingual_exact_match, 'xlingual_rougeL': xlingual_rougeL}
for metric_name, metric_var in metrics.items():
    metric_var.to_csv(metric_name+'_means.csv')


In [5]:
import json
import collections
import pandas as pd
import numpy as np
english_exp_names_1 = []
english_exp_names_3 = ['english_new_instruction']

exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)
classifications = ['answerability_classification', 'cause_effect_classification', 'coreference_resolution', \
    'dialogue_act_recognition', 'word_analogy', 'textual_entailment']
non_classifications = ['data_to_text', 'grammar_error_correction', 'title_generation',\
    'keyword_tagging', 'overlap_extraction', 'question_rewriting']
for english_exp_name in english_exp_names_1:
    metric_file = open('./tk_outputs/output_{}/all_results.json'.format(english_exp_name), 'r')
    metric = json.load(metric_file)


    for classification in classifications:
        exact_match[classification].append(metric['eval_exact_match_for_{}'.format(classification)])
    exact_match['eval_exact_match'].append(metric['eval_exact_match'])
    for non_classification in non_classifications:
        rougeL[non_classification].append(metric['eval_rougeL_for_{}'.format(non_classification)])
    rougeL['eval_rougeL'].append(metric['eval_rougeL'])

metric_file = open('./tk_outputs/output_{}/all_results.json'.format("english_original_instruction"), 'r')
metric = json.load(metric_file)
metric_names = metric.keys()
# print(metric)
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for english_exp_name in english_exp_names_3:
    metric_samples = create_dict(metric_names)
    metric_means = create_dict(metric_names)
    metric_stds = create_dict(metric_names)
    for index in [4,5,6]:
        _english_exp_name = english_exp_name + "_{}".format(index)

        metric_file = open('./tk_outputs/output_{}/all_results.json'.format(_english_exp_name), 'r')
        metric = json.load(metric_file)
        for metric_name in metric:
            if metric_name.endswith(f'_new_{index-3}'):
                metric_name_pruned = metric_name.replace(f'_new_{index-3}', '')
            else:
                metric_name_pruned = metric_name
            metric_samples[metric_name_pruned].append(metric[metric_name])
    
    for metric_name in metric_names:
        metric_means[metric_name] = round(np.mean(metric_samples[metric_name]), 1)
    for metric_name in metric_names:
        metric_stds[metric_name] = round(np.std(metric_samples[metric_name]), 1)

    # metrics = metric_means

    for classification in classifications:
        metric_mean = metric_means['eval_exact_match_for_{}'.format(classification)]
        metric_std = metric_stds['eval_exact_match_for_{}'.format(classification)]
        exact_match[classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    
    for non_classification in non_classifications:
        metric_mean = metric_means['eval_rougeL_for_{}'.format(non_classification)]
        metric_std = metric_stds['eval_rougeL_for_{}'.format(non_classification)]
        rougeL[non_classification].append(str(metric_mean) + "\u00B1" + str(metric_std))
    exact_match['eval_exact_match'].append(str(metric_means['eval_exact_match']) + "\u00B1" + str(metric_stds['eval_exact_match']))
    rougeL['eval_rougeL'].append(str(metric_means['eval_rougeL']) + "\u00B1" + str(metric_stds['eval_rougeL']))
english_rougeL = pd.DataFrame(rougeL)
english_rougeL.index = english_exp_names_1 + english_exp_names_3
english_exact_match = pd.DataFrame(exact_match)
english_exact_match.index = english_exp_names_1 + english_exp_names_3

def generate_new_columns(columns):
    new_columns = []
    for c in columns:
        new_column = []
        for first in c.split('_'):
            new_column.append(first[0].upper())
        new_columns.append("".join(new_column))
    return new_columns
english_exact_match.columns = generate_new_columns(english_exact_match.columns)
english_rougeL.columns = generate_new_columns(english_rougeL.columns)

In [6]:
print(english_exact_match.to_latex())
print(english_rougeL.to_latex())

\begin{tabular}{llllllll}
\toprule
{} &        AC &       CEC &        CR &       DAR &        WA &        TE &       EEM \\
\midrule
english\_new\_instruction &  65.8±2.2 &  67.3±2.9 &  53.0±1.9 &  44.5±7.0 &  42.3±3.1 &  49.3±4.9 &  36.5±1.0 \\
\bottomrule
\end{tabular}

\begin{tabular}{llllllll}
\toprule
{} &       DTT &       GEC &        TG &        KT &        OE &        QR &        ER \\
\midrule
english\_new\_instruction &  43.6±1.1 &  87.6±1.1 &  23.8±0.9 &  70.7±0.3 &  35.1±3.1 &  80.9±0.2 &  57.6±0.8 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_23280/2079488218.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_exact_match.to_latex())
/tmp/ipykernel_23280/2079488218.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(english_rougeL.to_latex())


\begin{tabular}{llllllll}
\toprule
{} &        AC &       CEC &        CR &       DAR &        WA &        TE &       EEM \\
\midrule
english\_original\_instruction &  69.5±1.9 &  69.2±0.6 &  58.5±2.1 &  48.7±3.1 &  44.5±0.4 &  44.8±2.1 &  37.4±0.5 \\
\bottomrule
\end{tabular}

\begin{tabular}{llllllll}
\toprule
{} &       DTT &       GEC &        TG &        KT &        OE &        QR &        ER \\
\midrule
english\_original\_instruction &  44.7±0.6 &  85.0±1.2 &  25.5±1.1 &  73.2±0.2 &  34.7±1.9 &  81.0±0.4 &  58.7±0.7 \\
\bottomrule
\end{tabular}

## test tk_instruct

In [7]:
import random
def random_shuffle(Definition):

    token_list = Definition.split()
    random.shuffle(token_list)
    return " ".join(token_list)
random_shuffle("I am a student")

'a am student I'

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("allenai/tk-instruct-3b-def")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/tk-instruct-3b-def")


In [18]:
input_ids = tokenizer.encode(
        # "Definition: return the currency of the country. Now complete the following example - Input: China. Output:", 
        "فتوسنتز فرایندی زیست‌شیمیایی است که در آن، انرژی نورانی خورشید توسط گیاهان و برخی از باکتری‌ها به انرژی شیمیایی ذخیره‌شده در مواد غذایی آن‌ها تبدیل می‌شود. کمابیش همهٔ جانداران روی زمین به آن وابسته‌اند. در عمل فتوسنتز، اندام‌هایی مانند برگ که دارای سبزینه هستند، کربن دی‌اکسید، آب و نور را جذب کرده و به کلروپلاست می‌رسانند. طی واکنش‌هایی که درون کلروپلاست انجام می‌گیرد، این مواد به اکسیژن و کربوهیدرات‌ها تبدیل می‌شوند. همه اکسیژن کنونی موجود بر روی زمین، فراوردهٔ فتوسنتز است. برخی از کربوهیدرات‌های مهم تولیدشده مانند گلوکز، می‌توانند به دیگر مواد آلی، لیپیدها، نشاسته، سلولز و پروتئین تبدیل شوند که برای تبدیل‌شدن به پروتئین، نیاز به نیتروژن دارند. ژان باپتیست ون هلمونت، یکی از نخستین آزمایش‌های مربوط به فتوسنتز را انجام داد. همه بخش‌های سبزرنگ گیاه، قادر به انجام عمل فتوسنتز هستند. مادهٔ سبز موجود در گیاهان که سبزینه یا کلروفیل نام دارد، آغازکنندهٔ واکنش‌های فتوسنتز است. فتوسنتز در اندام‌هایی که فاقد سبزینه هستند، انجام نمی‌گیرد.",
        return_tensors="pt"
    )
output = model.generate(input_ids, max_length=10)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

## load ni_dataset_perturb

In [2]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/induction_dataset.py", 
    data_dir='../induction_data/', 
    task_dir='../induction_data/tasks/', 
    cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    download_mode = 'reuse_cache_if_exists'
)
print(raw_datasets['test'][1])

Using custom data configuration default-ef21323b704ac40a


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset natural_instructions downloaded and prepared to ../cache/natural_instructions/default-ef21323b704ac40a/2.0.0/241d69c989a9550d22fa88048671e911038842f20f900bbc514909343345ccac. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'Task': "['translation', 'en-es']", 'Contributors': 'instruction_induction', 'Source': ['instruction_induction'], 'URL': ['instruction_induction'], 'Categories': ['instruction_induction'], 'Reasoning': ['instruction_induction'], 'Definition': ['Translate to Spanish'], 'Positive Examples': [], 'Negative Examples': [], 'Input_language': ['English'], 'Output_language': ['English'], 'Instruction_language': ['English'], 'Domains': ['instruction_induction'], 'Instance': {'input': 'ile', 'output': ['montón']}}


In [1]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/ni_dataset.py", 
    data_dir='../data/splits/default', 
    task_dir='../data/tasks', 
    # cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    # download_mode = 'reuse_cache_if_exists'
    download_mode="force_redownload"
)
print(raw_datasets['validation'][101]['Definition'])

Using custom data configuration default-8e82d9b055b27d6b


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset natural_instructions downloaded and prepared to /home/xxx/.cache/huggingface/datasets/natural_instructions/default-8e82d9b055b27d6b/2.0.0/c0040095172c0d76abd173b8640d8a5c5293cbc538c41e16bb88bdb4d0bb6a46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

["Read the passage and find the corresponding pronoun for the given name. The pronoun should match the given blank(_). The word between ** ** is the target name. The pronoun should be one of 'her', 'him', 'he', 'she' and 'his' with proper casing based on the position in the passage."]


In [2]:
from datasets import load_dataset, load_metric
from transformers import set_seed
set_seed(0)
raw_datasets = load_dataset(
    "../src/ni_dataset_perturb.py", 
    data_dir='../data/splits/default', 
    task_dir='../data/tasks', 
    cache_dir='../cache/',
    max_num_instances_per_task=100,
    max_num_instances_per_eval_task=100,
    perturb_method='replace_words',
    download_mode = 'reuse_cache_if_exists'
)

Using custom data configuration default-ec6c0847c0c95340


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Definition_native:  Write a suitable title for the text you see, containing the subject of this text, preferably in fifteen words or less.
Definition_change:  Write a subject subject for this text you see. the the subject for this text, written in fifteen words or more.
Definition_native:  Read the passage and find its corresponding pronoun for the target name. The position of the pronoun is at the given blank (_). The word between ** ** is the target name. The pronoun should be one of "she", "he", "him", "she" and "he". Appropriate case is required depending on where the pronoun is located.
Definition_change:  Read the passage and find the corresponding pronoun for the target name. The position of the pronoun is at the given blank ( right ). The position between * * * * is the target name. The pronoun should be one of " he ", " he ", " him ", " she " and " he ". Appropriate case is required depending on where the pronoun is located.
Definition_native:  Determine whether the content of

0 examples [00:00, ? examples/s]

Dataset natural_instructions downloaded and prepared to ../cache/natural_instructions/default-ec6c0847c0c95340/2.0.0/fb2cebb5e38344990106cf3b5c18496b4bf21e03c6d6a88d149d8279e4a82da3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

['In this task your given two statements in Estonian. You must judge whether the second sentence is the cause or effect of the first one. Label the instances as "cause" or "effect" based on your judgment. The sentences are separated by a newline character.']

## Data Perturbation

In [3]:
import numpy as np
import spacy
import random
from typing import Dict
from tqdm import tqdm
import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM

transformers.logging.set_verbosity_error()


In [4]:
class DataPerturbation:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
        self.en = spacy.load('en_core_web_sm')
        
    @staticmethod
    def connect_token_segments(tokens):
        connected_tokens = []
        for token in tokens:
            if token.startswith("##"):
                connected_tokens[-1] = connected_tokens[-1] + token[2:]
            else:
                connected_tokens.append(token)
        return connected_tokens


    def delete_words(self, Definition, num=5):

        tokens = self.tokenizer.tokenize(Definition)
        tokens = self.connect_token_segments(tokens)

        index = [i for i in range(len(tokens))]

        deleted_index = random.sample(index, num)
        deleted_index = set(deleted_index)

        deleted_tokens = [tokens[i] for i in index if i not in deleted_index]
        Definition_perturb = self.tokenizer.convert_tokens_to_string(deleted_tokens)
        return Definition_perturb
        
    def delete_stopwords(self, Definition):
        
        tokens = self.tokenizer.tokenize(Definition)
        tokens = self.connect_token_segments(tokens)

        stopwords = self.en.Defaults.stop_words
        deleted_tokens=[]
        for token in tokens:
            if token.lower() not in stopwords:
                deleted_tokens.append(token)
        Definition_perturb =  self.tokenizer.convert_tokens_to_string(deleted_tokens)
        return Definition_perturb

    def insert_words(self, Definition, num_mask=5):

        tokens = self.tokenizer.tokenize(Definition)
        if len(tokens)>512:
            return Definition
        tokens = self.connect_token_segments(tokens)

        index = [i for i in range(len(tokens))]

        index = random.sample(index, num_mask)

        for i in index:
            tokens.insert(i, '[MASK]')

        
        
        Definition = self.tokenizer.convert_tokens_to_string(tokens)
        inputs = self.tokenizer(Definition, return_tensors='pt')
        input_ids = inputs['input_ids'][0]
        outputs = self.model(**inputs)
        predictions = outputs[0]

        _, sorted_idx = predictions[0].sort(dim=-1, descending=True)

        predicted_index = [sorted_idx[i, 0].item() for i in range(0, len(predictions[0])-1)]
        for x in range(1, len(predictions[0])-1):
            if input_ids[x] == 103:
                input_ids[x] = predicted_index[x]

        return self.tokenizer.decode(input_ids, skip_special_tokens=True)

    def replace_words(self, Definition, num_mask=5):

        tokens = self.tokenizer.tokenize(Definition)
        if len(tokens)>512:
            return Definition
        tokens = self.connect_token_segments(tokens)

        index = [i for i in range(len(tokens))]

        index = random.sample(index, num_mask)

        for i in index:
            tokens[i] = '[MASK]'

        if len(tokens)>512:
            return Definition
        
        Definition = self.tokenizer.convert_tokens_to_string(tokens)
        inputs = self.tokenizer(Definition, return_tensors='pt')
        input_ids = inputs['input_ids'][0]
        outputs = self.model(**inputs)
        predictions = outputs[0]

        _, sorted_idx = predictions[0].sort(dim=-1, descending=True)

        predicted_index = [sorted_idx[i, 0].item() for i in range(0, len(predictions[0])-1)]
        for x in range(1, len(predictions[0])-1):
            if input_ids[x] == 103:
                input_ids[x] = predicted_index[x]

        return self.tokenizer.decode(input_ids, skip_special_tokens=True)
    
    def shuffle_words(self, Definition):

        tokens = self.tokenizer.tokenize(Definition)
        tokens = self.connect_token_segments(tokens)

        random.shuffle(tokens)
        return self.tokenizer.convert_tokens_to_string(tokens)
    
    def shuffle_sentences(self, Definition):

        doc = self.en(Definition)
        sents = list(map(str, doc.sents))
        random.shuffle(sents)
        return " ".join(sents)

    def repeat_sentences(self, Definition, index = None):
        doc = self.en(Definition)
        sents = list(map(str, doc.sents))
        if None == index:
            index = random.randint(0, len(sents)-1)
        sents = sents[:index] + [sents[index]] + sents[index:]
        return " ".join(sents)

In [5]:
data_perturbation = DataPerturbation()
sentence = "The quick brown fox jumps over the lazy dog."
print(data_perturbation.delete_words(sentence, 2))
print(data_perturbation.delete_stopwords(sentence))
print(data_perturbation.insert_words(sentence, 2))
print(data_perturbation.replace_words(sentence, 2))
print(data_perturbation.shuffle_words(sentence))
print(data_perturbation.shuffle_sentences(sentence))
print(data_perturbation.repeat_sentences(sentence))

quick fox jumps over the lazy dog .
quick brown fox jumps lazy dog .
The quick brown - fox " jumps over the lazy dog.
The quick dog fox jumps over the big dog.
. The lazy dog fox jumps brown the quick over
The quick brown fox jumps over the lazy dog.
The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.


In [6]:
import inspect
import pandas as pd
pd.set_option('display.max_colwidth', None)
original_instruction = "You are given a sentence from a conversation between a human and a virtual assistant. Your task is to classify the sentence into one of the following five action categories - INFORM, INFORM_INTENT, OFFER, REQUEST, REQUEST_ALTS. If the sentence is about informing something, generate 'INFORM'. If it is about requesting something, generate 'REQUEST'. If the sentence mentions requesting an alternative option than the one provided, generate 'REQUEST_ALTS'. Similarly, generate 'OFFER' if the sentence is offering some information. If the sentence is about showing intent to do something, generate 'INFORM_INTENT'."

data_perturbation = DataPerturbation()
attrs = (getattr(data_perturbation, name) for name in dir(data_perturbation))
methods = filter(inspect.ismethod, attrs)
compares = [["Original Instruction", original_instruction]]
for method in methods:
    if method.__name__ != '__init__':
        compares.append([method.__name__, method(original_instruction)])

df_compares = pd.DataFrame(compares)
# print(df_compares.style.to_latex(hrules=True))
print(df_compares)

                      0  \
0  Original Instruction   
1      delete_stopwords   
2          delete_words   
3          insert_words   
4      repeat_sentences   
5         replace_words   
6     shuffle_sentences   
7         shuffle_words   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       1  
0                                                                   You a

In [1]:
import glob
jsons = glob.glob('./data/tasks/*.json')

In [5]:
import json
for j in jsons:
    task_j = json.load(open(j))
    if task_j['Instruction_language'][0] != 'English':
        print(task_j['Instruction_language'])
    